In [ ]:
import numpy as np
import librosa, librosa.display 
import matplotlib.pyplot as plt

In [ ]:
audio, sr = librosa.load("/content/drive/MyDrive/해커톤/녹음.wav", sr=16000)
print('sr:', sr, ', audio shape:', audio.shape)
print('length:', audio.shape[0]/float(sr), 'secs')

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


sr: 16000 , audio shape: (143019,)
length: 8.9386875 secs


In [ ]:
mfcc = librosa.feature.mfcc(audio, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)

In [ ]:
import sklearn
mfcc = sklearn.preprocessing.scale(mfcc, axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:173: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:190: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [ ]:
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))
padded_mfcc = pad2d(mfcc, 800)
padded_mfcc= np.expand_dims(padded_mfcc, 0)

In [99]:
!pip install pydub

In [134]:
from pydub.silence import detect_nonsilent
min_silence_length = 1000
intervals = detect_nonsilent("/content/drive/MyDrive/해커톤/music2.wav", min_silence_len=min_silence_length, silence_thresh=200000)

In [135]:
print(intervals)
print("start,Stop")
for chunks in intervals:
    print( [chunk/1000 for chunk in chunks])

[[0, 41]]
start,Stop
[0.0, 0.041]


In [ ]:
def shorter_filler(json_result, audio_file, min_silence_len, start_time, non_silence_start):
  
  # 침묵 길이를 더 짧게
  min_silence_length = (int)(min_silence_len/1.2)

  intervals = detect_nonsilent(audio_file,
                              min_silence_len=min_silence_length,
                              silence_thresh=-32.64
                              )
  
  for interval in intervals:

    interval_audio = audio_file[interval[0]:interval[1]]

    # padding 40 길이 이상인 경우 더 짧게
    if (interval[1]-interval[0] >= 460):
      non_silence_start = shorter_filler(json_result, interval_audio, min_silence_length, interval[0]+start_time, non_silence_start)

    else: # padding 40 길이보다 짧은 경우 predict
      if predict_filler(interval_audio) == 0 : # 추임새인 경우
        json_result.append({'start':non_silence_start,'end':start_time+interval[0],'tag':'1000'})
        non_silence_start = start_time + interval[0]
        
        # 추임새 tagging
        json_result.append({'start':start_time+interval[0],'end':start_time+interval[1],'tag':'1111'}) 
        
    
  return non_silence_start

In [ ]:
def create_json(audio_file):
  intervals_jsons = []

  min_silence_length = 70
  intervals = detect_nonsilent(audio_file,
                              min_silence_len=min_silence_length,
                              silence_thresh=-1.64
                              )
  
  if intervals[0][0] != 0:
    intervals_jsons.append({'start':0,'end':intervals[0][0],'tag':'0000'}) 
    
  non_silence_start = intervals[0][0]
  before_silence_start = intervals[0][1]

  for interval in intervals:
    interval_audio = audio_file[interval[0]:interval[1]]

     # 800ms초 이상의 공백 부분 처리
    if (interval[0]-before_silence_start) >= 800:
      intervals_jsons.append({'start':non_silence_start,'end':before_silence_start+200,'tag':'1000'}) 
      non_silence_start = interval[0]-200
      intervals_jsons.append({'start':before_silence_start,'end':interval[0],'tag':'0000'}) 


    if len(interval_audio) <= 460:
      intervals_jsons.append({'start':non_silence_start,'end':interval[0],'tag':'1000'})
      non_silence_start = interval[0]
      intervals_jsons.append({'start':interval[0],'end':interval[1],'tag':'1111'})
    else:
      non_silence_start = shorter_filler(intervals_jsons, interval_audio, min_silence_length, interval[0], non_silence_start)
    
    before_silence_start = interval[1]

  if non_silence_start != len(audio_file):
    intervals_jsons.append({'start':non_silence_start,'end':len(audio_file),'tag':'1000'})

  return intervals_jsons

In [ ]:
abc = create_json("/content/drive/MyDrive/해커톤/녹음.wav")

In [ ]:
abc

[{'end': 0, 'start': 0, 'tag': '1000'},
 {'end': 37, 'start': 0, 'tag': '1111'},
 {'end': 37, 'start': 0, 'tag': '1000'}]

In [ ]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
#adjust target amplitude
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)
#Convert wav to audio_segment

audio_segment, sr = librosa.load("/content/drive/MyDrive/해커톤/녹음.wav", sr=16000)
print('sr:', sr, ', audio shape:', audio.shape)
print('length:', audio.shape[0]/float(sr), 'secs')

#normalize audio_segment to -20dBFS 
normalized_sound = match_target_amplitude("/content/drive/MyDrive/해커톤/녹음.wav", -20.0)
print("length of audio_segment={} seconds".format(len(normalized_sound)/1000))
#Print detected non-silent chunks, which in out case would be spoken words.
nonsilent_data = detect_nonsilent(normalized_sound, min_silence_len=50, silence_thresh=5000)
#convert ms to seconds
print("start,Stop")
for chunks in nonsilent_data:
    print( [chunk/1000 for chunk in chunks])

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


sr: 16000 , audio shape: (143019,)
length: 8.9386875 secs


AttributeError: ignored

In [ ]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
#adjust target amplitude
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)
#Convert wav to audio_segment
audio_segment = AudioSegment.from_wav("/content/drive/MyDrive/해커톤/녹음.wav")
print(audio_segment)
#normalize audio_segment to -20dBFS 
normalized_sound = match_target_amplitude(audio_segment, -20)
print("length of audio_segment={} seconds".format(len(normalized_sound)/1000))
#Print detected non-silent chunks, which in out case would be spoken words.
nonsilent_data = detect_nonsilent(normalized_sound, min_silence_len=500, silence_thresh=-20, seek_step=1)
#convert ms to seconds
print("start,Stop")
for chunks in nonsilent_data:
    print( [chunk/1000 for chunk in chunks])

CouldntDecodeError: ignored

In [ ]:
!pip install FFmpeg

  Created wheel for FFmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=212a56aa569a7caacb38948c47baaef4128c6259f042d10f7aadd73a21b09805
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built FFmpeg
